Домашнее задание по теме "Коллаборативная фильтрация" (№2)

ПАКЕТ SURPRISE

- используйте данные MovieLens 1M
- можно использовать любые модели из пакета
- получите RMSE на тестовом сете 0.87 и ниже

Комментарий преподавателя :
В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.

In [80]:
# Попробуем всё-таки решить задачу на датасете в 1 млн.рейтингов. 
# (MovieLens 1M movie ratings. Stable benchmark dataset. 1 million ratings from 6000 users on 4000 movies. Released 2/2003.)
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip

--2022-08-07 16:22:58--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

ml-1m.zip.1         100%[===================>]   5.64M  18.6MB/s    in 0.3s    

2022-08-07 16:22:58 (18.6 MB/s) - ‘ml-1m.zip.1’ saved [5917549/5917549]



In [81]:
!unzip ml-1m.zip

Archive:  ml-1m.zip
replace ml-1m/movies.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [82]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook

In [83]:
!pip3 install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [84]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

In [85]:
movies = pd.read_csv('/content/ml-1m/movies.dat',
                        delimiter='::', engine= 'python', header=None, encoding='latin-1',
                        names=['movie_id', 'movie_name', 'genre'])

In [86]:
movies.head(3)

,movie_id,movie_name,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [87]:
ratings = pd.read_csv('/content/ml-1m/ratings.dat',
                        delimiter='::', engine= 'python', header=None, encoding='latin-1',
                        names=['user_id', 'movie_id', 'rating', 'time'])

In [88]:
ratings.head(3)

,user_id,movie_id,rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968


In [89]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype
---  ------    --------------    -----
 0   user_id   1000209 non-null  int64
 1   movie_id  1000209 non-null  int64
 2   rating    1000209 non-null  int64
 3   time      1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


In [90]:
movies_with_ratings = movies.join(ratings.set_index('movie_id'), on='movie_id').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [91]:
movies_with_ratings.head(3)

,movie_id,movie_name,genre,user_id,rating,time
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,5.0,978824268.0
1,1,Toy Story (1995),Animation|Children's|Comedy,6.0,4.0,978237008.0
2,1,Toy Story (1995),Animation|Children's|Comedy,8.0,4.0,978233496.0


In [92]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.user_id,
    'iid': movies_with_ratings.movie_name,
    'rating': movies_with_ratings.rating
})

In [93]:
dataset.head(3)

,uid,iid,rating
0,1.0,Toy Story (1995),5.0
1,6.0,Toy Story (1995),4.0
2,8.0,Toy Story (1995),4.0


In [94]:
ratings.rating.min()

1

In [95]:
ratings.rating.max()

5

In [96]:
reader = Reader(rating_scale=(1.0, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [97]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=21)

In [98]:
algo = KNNWithMeans(k=30, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [99]:
test_pred = algo.test(testset)

In [100]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8872


0.8871509681309035

## На отложенной (трейн/тест) выборке RMSE не удовлетворяет условиям задачи (должно быть 0.87 и менее). Попробуем кросс-валидацию (на 5-и фолдах), алгоритм пока возьмем тот же (KNNWithMeans).

In [101]:
from surprise.model_selection import cross_validate

In [102]:
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True, n_jobs=-1)

Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8878  0.8911  0.8908  0.8884  0.8935  0.8903  0.0021  
Fit time          107.88  116.79  126.13  115.71  73.20   107.94  18.31   
Test time         146.08  171.57  151.23  140.58  68.59   135.61  35.11   


{'fit_time': (107.87824892997742,
  116.79031181335449,
  126.13152265548706,
  115.71186065673828,
  73.20266962051392),
 'test_rmse': array([0.88784385, 0.89112596, 0.89075578, 0.88840849, 0.89354862]),
 'test_time': (146.07787442207336,
  171.56840419769287,
  151.22883415222168,
  140.5772671699524,
  68.58715033531189)}

## К сожалению, ни один вариант разбивки на фолды с алгоритмом KNNWithMeans не позволил снизить ошибку до 0.87 и менее. Попробуем другие алгоритмы коллаборативной фильтрации.

In [103]:
# Возьмем базовый алгоритм коллаборативной фильтрации - KNNBasic
from surprise import KNNBasic

In [104]:
algo = KNNBasic(k=30, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [105]:
test_pred = algo.test(testset)

In [106]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9236


0.9236276226160715

## к сожалению, RMSE увеличилась по сравнению с KNNWithMeans. Попробуем другую группу алгоритмов: Matrix Factorization-based algorithms

In [107]:
# The SVD++ algorithm, an extension of SVD taking into account implicit ratings.
from surprise import SVDpp

In [108]:
# используем параметры по умолчанию
algo = SVDpp()
algo.fit(trainset)

In [109]:
test_pred = algo.test(testset)

In [110]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8595


0.8594985423174889

## Как видим, применение алгоритма SVD (его "продвинутой" версии SVD++) позволило уменьшить RMSE до значения 0.8595. Однако этот алгоритм требует существенно больше времени на расчеты, чем KNN. Следует отметить, что RMSE на таком высоком уровне не является хорошим результатом. Видимо, нужна предобработка данных (Feature engineering) перед использованием библиотеки Surprise.